## Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Setting to make number easier to ready on display
pd.options.display.float_format = '{:20.2f}'.format

# Show all columns on output
pd.set_option('display.max_columns', 999)

## Data Exploration

In [4]:
df = pd.read_excel('./Data/online_retail_II.xlsx')

df.head(10)

ValueError: Invalid format specifier '20:2f' for object of type 'float'

ValueError: Invalid format specifier '20:2f' for object of type 'float'

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [6]:
df.describe()

ValueError: Invalid format specifier '20:2f' for object of type 'float'

ValueError: Invalid format specifier '20:2f' for object of type 'float'

In [7]:
df.describe(include='0')

TypeError: data type '' not understood

In [ ]:
df[df["Customer ID"].isna()].head(10)

In [ ]:
df[df["Quantity"] < 0].head(10)

In [ ]:
df["Invoice"] = df["Invoice"].astype("str")
df[df["Invoice"].str.match("^\\d{6}$") == False]